## Google Restaurant Reviews and Suggestions

### Environment Setup
Install packages.
>Secrets (API keys) **ARE USED** in this demo.

To use this code you'll need to upload a `.env` file to the root directory for this notebook. Click on `File Browser` (folder) icon to view the directory. All files starting with a period are hidden by default. Click the eye icon to view them.

View the [README](https://console.cloud.google.com/vertex-ai/colab/notebooks?project=vertexaiwebinar&supportedpurview=project&activeNb=projects%2Fvertexaiwebinar%2Flocations%2Fus-central1%2Frepositories%2Fccc4213c-e5f1-4017-a1ef-f1984b49b4dc) for information on how to obtain and use your own API keys from an `.env` file.


In [1]:
!pip install --upgrade google-cloud-aiplatform
!pip install python-dotenv

### Look Up Restaurants and Send to Text-Unicorn
Call the Google Places API

Uses the `text-unicorn@001` model to generate a summarized list with the higest rated at the top.

In [6]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

# Used to pull in API Keys/Secrets
from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()
api_key = os.getenv("api_key")
project_id = os.getenv("project_id")

import requests
import math
import vertexai

from vertexai.preview.language_models import TextGenerationModel

# Define the search radius and other parameters
location_name = 'Erie, Co'  # Replace with the city or town of your choice
radius = '20000'  # 20000 is approx 20 miles
place_type = 'restaurant'
keyword = 'mexican'

# Haversine function to calculate distance
def haversine(lat1, lon1, lat2, lon2):
    """Calculate the great circle distance in kilometers between two points
    on the earth (specified in decimal degrees)"""
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 3956  # Radius of earth in kilometers. Use 3956 for miles
    return c * r

# Function to convert a location into latitude and longitude
def get_location_coordinates(api_key, location):
    geocode_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={location}&key={api_key}"
    response = requests.get(geocode_url)
    data = response.json()
    if 'results' in data and len(data['results']) > 0:
        latitude = data['results'][0]['geometry']['location']['lat']
        longitude = data['results'][0]['geometry']['location']['lng']
        return f"{latitude},{longitude}"
    else:
        return None

# Get the latitude and longitude for the location
location_coordinates = get_location_coordinates(api_key, location_name)
if not location_coordinates:
    raise Exception("Failed to get location coordinates. Check your location name and API key.")

# Split the coordinates into latitude and longitude for distance calculation
location_lat, location_lon = map(float, location_coordinates.split(','))

# Endpoint URL for the Nearby Search request
url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location_coordinates}&radius={radius}&type={place_type}&keyword={keyword}&key={api_key}"

# Make a GET request to the Google Maps API
response = requests.get(url)
data = response.json()

# Collecting restaurant information
restaurant_details_string = ""

if data.get('results'):
    for place in data['results']:
        name = place.get('name', 'No name available')
        address = place.get('vicinity', 'No address available')
        rating = place.get('rating', 'No rating available')
        place_id = place.get('place_id', 'No place ID available')  # Extract place_id
        open_now = 'No information available'
        if 'opening_hours' in place and 'open_now' in place['opening_hours']:
            open_now = 'Yes' if place['opening_hours']['open_now'] else 'No'
        place_lat = place['geometry']['location']['lat']
        place_lon = place['geometry']['location']['lng']
        distance = haversine(location_lat, location_lon, place_lat, place_lon)

        # Append restaurant information to the string
        restaurant_details_string += f"Name: {name}, Address: {address}, Rating: {rating}, Open Now: {open_now}, Distance: {distance:.2f} miles, , Place ID: {place_id}\n\n"
else:
    print("No nearby restaurants found or an error occurred.")

#paste in reviews
def get_place_details(api_key, place_id):
    """Fetch place details for a given place_id from the Google Places API."""
    details_url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={api_key}"

    response = requests.get(details_url)
    data = response.json()

    if data['status'] == 'OK':
        # Extract and print details
        details = data['result']
        name = details.get('name', 'No name available')
        address = details.get('formatted_address', 'No address available')
        phone = details.get('formatted_phone_number', 'No phone number available')
        website = details.get('website', 'No website available')
        place_id = place.get('place_id', 'No place ID available')  # Extract place_id

        print(f"Name: {name}")
        print(f"Address: {address}")
        print(f"Phone Number: {phone}")
        print(f"Website: {website}")

# Prepare the content for the language model
prompt = "You are a local food afficienado. I need your help choosing a restaurant to visit. Here are some local restaurants: "
print("#####################################")
print(prompt)
print(restaurant_details_string)
full_prompt = prompt + restaurant_details_string + "\n\n Summarize and order these restaurants by rating with the highest at the top. Additionally, write a few reasons why someone might choose the first 3 restaurants on the list and provide a corresponding URL for each restaurant's website."
print("#####################################")

def predict_large_language_model_sample(
    project_id: str,
    model_name: str,
    temperature: float,
    max_decode_steps: int,
    top_p: float,
    top_k: int,
    content: str,
    location: str = "us-central1",
    tuned_model_name: str = "",
    ):
    """Predict using a Large Language Model."""
    vertexai.init(project=project_id, location=location)
    model = TextGenerationModel.from_pretrained(model_name)
    if tuned_model_name:
        model = model.get_tuned_model(tuned_model_name)
    response = model.predict(
        content,
        temperature=temperature,
        max_output_tokens=max_decode_steps,
        top_k=top_k,
        top_p=top_p,)
    print("##### Vertex AI - text-unicorn Response ######")
    print("##### ------------------------------- ######")
    print(f"{response.text}")

# Call the function to use the language model
predict_large_language_model_sample(project_id, "text-unicorn@001", 0.15, 1000, 0.7, 30, full_prompt, "us-central1")


#####################################
You are a local food afficienado. I need your help choosing a restaurant to visit. Here are some local restaurants: 
Name: Tequila's Family Mexican Restaurant, Address: 12020 Pennsylvania St, Thornton, Rating: 4.2, Open Now: Yes, Distance: 10.03 miles, , Place ID: ChIJg_2L-DN0bIcRMwQn9Yp1oAo

Name: El Mirador Mexican Restaurant, Address: 11370 Huron St, Northglenn, Rating: 4.4, Open Now: Yes, Distance: 10.56 miles, , Place ID: ChIJWQxkTNd1bIcRLxnv6tU8zqs

Name: Hacienda Colorado, Address: 10422 Town Center Dr, Westminster, Rating: 4.2, Open Now: Yes, Distance: 11.35 miles, , Place ID: ChIJ6ZWIs4CJa4cRIv72q1jsgL0

Name: La Estrellita, Address: 45 N Main St, Brighton, Rating: 4.5, Open Now: Yes, Distance: 12.81 miles, , Place ID: ChIJZ6ZfmThtbIcRJo6l85no6co

Name: Los Jimadores, Address: 275 S 4th Ave, Brighton, Rating: 4.2, Open Now: Yes, Distance: 13.11 miles, , Place ID: ChIJQTCMOj5tbIcRke08jqvwKIU

Name: Casa Agave, Address: 451 S Pratt Pkwy, Lon

### Query Restaurant Reviews

In [7]:
import requests
import math
import os
from dotenv import load_dotenv

# Constants and Parameters
EARTH_RADIUS_MILES = 3956  # Radius of earth in miles
LOCATION_NAME = 'Erie, Co'  # Replace with the city or town of your choice
RADIUS = '20000'  # In meters (approx 20 miles)
PLACE_TYPE = 'restaurant'
KEYWORD = 'mexican'

# Load environment variables from .env file
load_dotenv()
api_key = os.getenv("api_key")
project_id = os.getenv("project_id")

# Utility Functions
def haversine(lat1, lon1, lat2, lon2):
    """Calculate the great circle distance between two points on the earth."""
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    return c * EARTH_RADIUS_MILES

def get_location_coordinates(api_key, location):
    """Convert a location into latitude and longitude."""
    geocode_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={location}&key={api_key}"
    response = requests.get(geocode_url)
    data = response.json()
    if 'results' in data and len(data['results']) > 0:
        lat = data['results'][0]['geometry']['location']['lat']
        lon = data['results'][0]['geometry']['location']['lng']
        return f"{lat},{lon}"
    return None

def get_place_details(api_key, place_id):
    """Fetch and print details for a given place_id from the Google Places API."""
    details_url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={api_key}"
    response = requests.get(details_url)
    data = response.json()

    if data['status'] == 'OK':
        details = data['result']
        print_place_details(details)
    else:
        print("Error fetching details:", data.get('status', 'Unknown error'))

def print_place_details(details):
    """Print details and reviews for a place."""
    # Basic place details
    name = details.get('name', 'No name available')
    address = details.get('formatted_address', 'No address available')
    phone = details.get('formatted_phone_number', 'No phone number available')
    website = details.get('website', 'No website available')
    rating = details.get('rating', 'No rating available')
    place_id = details.get('place_id', 'No place ID available')

    # Print basic details
    print(f"Name: {name}\nAddress: {address}\nPhone Number: {phone}\nWebsite: {website}\nRating: {rating}\nPlace ID: {place_id}\n")

    # Print reviews
    reviews = details.get('reviews', [])
    if reviews:
        print("Recent Reviews:")
        for review in reviews:
            print_review(review)

def print_review(review):
    """Print a single review."""
    author_name = review.get('author_name', 'Anonymous')
    rating = review.get('rating', 'No rating')
    text = review.get('text', '').replace('\n', ' ')
    time = review.get('relative_time_description', 'No time provided')

    print(f"Author: {author_name}\nRating: {rating}\nReview: {text}\nTime: {time}\n")
    print("\n-----------------\n")

# Main Script
location_coordinates = get_location_coordinates(api_key, LOCATION_NAME)
if not location_coordinates:
    raise Exception("Failed to get location coordinates. Check your location name and API key.")

# Endpoint URL for the Nearby Search request
url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location_coordinates}&radius={RADIUS}&type={PLACE_TYPE}&keyword={KEYWORD}&key={api_key}"

# Make a GET request to the Google Maps API for nearby places
response = requests.get(url)
data = response.json()

# Process and display results
if data.get('results'):
    for place in data['results']:
        place_id = place.get('place_id', '')
        if place_id:
            get_place_details(api_key, place_id)
else:
    print("No nearby places found or an error occurred.")


Name: Tequila's Family Mexican Restaurant
Address: 12020 Pennsylvania St, Thornton, CO 80241, USA
Phone Number: (303) 254-5555
Website: https://www.tequilasthornton.com/
Rating: 4.2
Place ID: ChIJg_2L-DN0bIcRMwQn9Yp1oAo

Recent Reviews:
Author: Peter Phakonekham
Rating: 5
Review: We came with a party of 6.  Food was great! For someone who’s not a huge fan of Mexican food I really enjoyed it to my liking! I got the steak dish, forgot the name. But I loved how the beef was cooked with the bell peppers! I would get it again and the service was very kind! The ice cream deep fried was super good!
Time: 2 months ago


-----------------

Author: Sammie
Rating: 5
Review: 1st time here!! It was very good. I liked the food & drinks we ordered!! The waiter was nice & they were very attentive! I enjoyed the meal! It was all very good. The drinks were very good & tasty! Overall bill was decent for what we ordered! They also do have a lot of tvs on throughout the restaurant so if your into sports mo

### Send Reviews to the Unicorn model for Summary

Get `place_id` of restaurants


In [8]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

# Used to pull in API Keys/Secrets
from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()
api_key = os.getenv("api_key")
project_id = os.getenv("project_id")

import requests
import math
import vertexai

from vertexai.preview.language_models import TextGenerationModel

# Define the search radius and other parameters
location_name = 'Erie, Co'  # Replace with the city or town of your choice
radius = '20000'  # 20000 is approx 20 miles
place_type = 'restaurant'
keyword = 'mexican'

# Haversine function to calculate distance
def haversine(lat1, lon1, lat2, lon2):
    """Calculate the great circle distance in kilometers between two points
    on the earth (specified in decimal degrees)"""
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 3956  # Radius of earth in kilometers. Use 3956 for miles
    return c * r

# Function to convert a location into latitude and longitude
def get_location_coordinates(api_key, location):
    geocode_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={location}&key={api_key}"
    response = requests.get(geocode_url)
    data = response.json()
    if 'results' in data and len(data['results']) > 0:
        latitude = data['results'][0]['geometry']['location']['lat']
        longitude = data['results'][0]['geometry']['location']['lng']
        return f"{latitude},{longitude}"
    else:
        return None

# Get the latitude and longitude for the location
location_coordinates = get_location_coordinates(api_key, location_name)
if not location_coordinates:
    raise Exception("Failed to get location coordinates. Check your location name and API key.")

# Split the coordinates into latitude and longitude for distance calculation
location_lat, location_lon = map(float, location_coordinates.split(','))

# Endpoint URL for the Nearby Search request
url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location_coordinates}&radius={radius}&type={place_type}&keyword={keyword}&key={api_key}"

# Make a GET request to the Google Maps API
response = requests.get(url)
data = response.json()

# Collecting restaurant information
restaurant_details_string = ""

if data.get('results'):
    for place in data['results']:
        name = place.get('name', 'No name available')
        address = place.get('vicinity', 'No address available')
        rating = place.get('rating', 'No rating available')
        place_id = place.get('place_id', 'No place ID available')  # Extract place_id
        open_now = 'No information available'
        if 'opening_hours' in place and 'open_now' in place['opening_hours']:
            open_now = 'Yes' if place['opening_hours']['open_now'] else 'No'
        place_lat = place['geometry']['location']['lat']
        place_lon = place['geometry']['location']['lng']
        distance = haversine(location_lat, location_lon, place_lat, place_lon)

        # Append restaurant information to the string
        restaurant_details_string += f"Name: {name}, Address: {address}, Rating: {rating}, Open Now: {open_now}, Distance: {distance:.2f} miles, , Place ID: {place_id}\n\n"
else:
    print("No nearby restaurants found or an error occurred.")

#paste in reviews
def get_place_details(api_key, place_id):
    """Fetch place details for a given place_id from the Google Places API."""
    details_url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={api_key}"
    response = requests.get(details_url)
    data = response.json()

    if data['status'] == 'OK':
        # Extract and print details
        details = data['result']
        name = details.get('name', 'No name available')
        place_id = details.get('place_id', 'No place ID available')

    else:
        print("Error fetching details:", data.get('status', 'Unknown error'))

# Main Script
def main():
    location_coordinates = get_location_coordinates(api_key, location_name)
    if not location_coordinates:
        raise Exception("Failed to get location coordinates. Check your location name and API key.")

    location_lat, location_lon = map(float, location_coordinates.split(','))

    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location_coordinates}&radius={radius}&type={place_type}&keyword={keyword}&key={api_key}"
    response = requests.get(url)
    data = response.json()

    if data.get('results'):
        for place in data['results']:
            name = place.get('name', 'No name available')
            place_id = place.get('place_id', 'No place ID available')

            # Print restaurant information
            print(f"Name: {name} :: Place ID: {place_id}\n")

            # Fetch and print details for each place_id
            get_place_details(api_key, place_id)
    else:
        print("No nearby restaurants found or an error occurred.")

if __name__ == "__main__":
    main()

Name: Tequila's Family Mexican Restaurant :: Place ID: ChIJg_2L-DN0bIcRMwQn9Yp1oAo

Name: El Mirador Mexican Restaurant :: Place ID: ChIJWQxkTNd1bIcRLxnv6tU8zqs

Name: Hacienda Colorado :: Place ID: ChIJ6ZWIs4CJa4cRIv72q1jsgL0

Name: La Estrellita :: Place ID: ChIJZ6ZfmThtbIcRJo6l85no6co

Name: Los Jimadores :: Place ID: ChIJQTCMOj5tbIcRke08jqvwKIU

Name: Casa Agave :: Place ID: ChIJGfokW5f5a4cRB5ofRjjXl6Y

Name: Rio Grande Mexican Restaurant :: Place ID: ChIJJU2aYSbsa4cRJC5YrD0nPFs

Name: 3 Margaritas :: Place ID: ChIJlUT1r0ZxbIcRC9Oq2QiuSoU

Name: El Aguascalientes Mexican Restaurant :: Place ID: ChIJHVe2cWttbIcRC5hWYyL6jS0

Name: Santiago's Mexican Restaurant :: Place ID: ChIJn3qyZWb5a4cRLBRwz_KhWmY

Name: El Tapatio Mexican Restaurant :: Place ID: ChIJTR_ea4WMa4cRJequlENJNzQ

Name: El Azteca Mexican Food :: Place ID: ChIJe0T0bhd0bIcR1qrSV_wWeWc

Name: Lupita's Mexican Restaurant :: Place ID: ChIJV_x1NsltbIcR6b3lfXmciwA

Name: 3 Margaritas Longmont :: Place ID: ChIJ_ZyT_zX5a4cR2zEeF

In [9]:
import requests
import vertexai
from vertexai.preview.language_models import TextGenerationModel
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()
api_key = os.getenv("api_key")
project_id = os.getenv("project_id")

# Constants
MODEL_NAME = 'text-unicorn@001'  # Replace with the actual model name you have access to
PLACE_ID = 'ChIJHVe2cWttbIcRC5hWYyL6jS0'  # Replace with the place_id for the restaurant you're interested in

# Function Definitions
def get_place_reviews(api_key, place_id):
    """
    Fetch and return all reviews for a given place.

    Parameters:
        api_key (str): API key for Google Places API.
        place_id (str): The place ID to fetch reviews for.

    Returns:
        str: A string containing all reviews concatenated together.
    """
    details_url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=name,rating,reviews&key={api_key}"
    response = requests.get(details_url)
    data = response.json()
    reviews = []

    if data['status'] == 'OK':
        for review in data['result'].get('reviews', []):
            text = review.get('text', '').replace('\n', ' ')
            reviews.append(text)
        all_reviews = ' '.join(reviews)  # Concatenate all reviews into a single text

        # Print all reviews for verification
        print("All Reviews:")
        for i, review in enumerate(reviews, 1):
            print(f"{i}: {review}\n")

        return all_reviews
    else:
        print("Error fetching details:", data.get('status', 'Unknown error'))
        return ""

def summarize_reviews(project_id, model_name, reviews):
    """
    Summarize the provided reviews using Text-Bison model.

    Parameters:
        project_id (str): Project ID for Vertex AI.
        model_name (str): The name of the model to use for generation.
        reviews (str): A string containing all reviews to summarize.
    """
    prompt = "Using the following reviews of this restaurant, generate a summary review that encompasses all opinions. Include a quote from one of the reviews, including the author's name. Reviews: "
    full_content = prompt + reviews  # Prepend the prompt to the reviews

    vertexai.init(project=project_id)
    model = TextGenerationModel.from_pretrained(model_name)
    response = model.predict(full_content, temperature=0.7, max_output_tokens=150)

    # Print the summarized reviews
    print("Summarized Reviews:")
    print(response.text)

# Main Script
def main():
    all_reviews = get_place_reviews(api_key, PLACE_ID)
    if all_reviews:
        summarize_reviews(project_id, MODEL_NAME, all_reviews)
    else:
        print("No reviews available to summarize.")

if __name__ == "__main__":
    main()


All Reviews:
1: The food was amazing the day we went the serive was great. And the servings are huge. My wife had the carnitas and I had a stake chimichanga.

2: Just stopped in and needed a few handheld burritos early on a Sunday morning ..reasonable prices and delicious burritos! The lady at the front was very nice. A little longer wait but that was because they were making things fresh and grilling the tortilla. Thanks!

3: No doubt that the food is good, and you get a great portion for a great price. But the service is not so great they take forever to get you anything or to do anything and not overly friendly and it smells really bad like sewer in there. There is very crowded. I think this facility should move somewhere else where it could do so much better, the smell of sewer is disgusting

4: Gave this place a shot as we were driving through, we are happy that we did. Food was great, service was quick and very helpful, atmosphere was well done, plenty of menu options for everyon